# Block 6 Exercise 2: finding the best parameters for predicting the fare of taxi rides
We return to our Random Forest Regression and want to automatically optimize all free parameters ...

In [0]:
import pandas as pd
import numpy as np
import folium


In [0]:
#check if notebook runs in colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print('running in Colab:',IN_COLAB)
path='..'
if IN_COLAB:
  #in colab, we need to clone the data from the repo
  !git clone https://github.com/keuperj/DataScienceSS20.git
  path='DataScienceSS20'

running in Colab: True
Cloning into 'DataScienceSS20'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 572 (delta 5), reused 17 (delta 5), pack-reused 550
Receiving objects: 100% (572/572), 130.60 MiB | 25.27 MiB/s, done.
Resolving deltas: 100% (226/226), done.
Checking out files: 100% (217/217), done.


In [0]:
# we load the data we have saved after wrangling and pre-processing in block I
X=pd.read_csv(path+'/DATA/train_cleaned.csv')
drop_columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','key','pickup_datetime','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
X=X.drop(drop_columns,axis=1)
X=pd.get_dummies(X)# one hot coding
#generate labels
y=X['fare_amount']
X=X.drop(['fare_amount'],axis=1)

### Scikit Optimize
Scikit Optimize (https://scikit-optimize.github.io/stable/index.html) is a AutoML toolbox wrapped around Scikit-Learn. It allows us to use state-of-the-art automatic hyper-parameter optimization on top of our learning algorithms.   



In [0]:
# install 
!pip install scikit-optimize

     |████████████████████████████████| 81kB 3.1MB/s 


### E 2.1 Bayesian Optimization of a Random Forest Regression Model
use Bayesian Optimization with Cross-Validation (https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html#skopt.BayesSearchCV) to find the best regression model. Compare
* linear regression (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) 
* Random Forest regression (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)
* and SVM regression (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR)

NOTES: this can become quite compute intensive! Hence,
* use a smaller subset of the training data to run the experiments 
* think about the range of your parameters (e.g. larger number of trees in RF or high C-values in SMV will make models expensive)
* optimize only the following parameters per model type:
    * linear: no parameters to optimize
    * RF: #trees and depth
    * SVM: C and gamma (use RBF kernel)
* parallelize -> n_jobs
* use CoLab to rum the job for up to 12h 


In [0]:
print (np.shape(X), " and ", np.shape(y))

(400000, 31)  and  (400000,)


In [0]:
X_subset = X[0:20000].astype(np.float64)
y_subset = y[0:20000].astype(np.float64)

X_test = X[20000:30000].astype(np.float64)
y_test = y[20000:30000].astype(np.float64)

In [0]:
print(np.shape(X_subset), " and ", np.shape(y_subset))

(20000, 31)  and  (20000,)


In [0]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_subset, y_subset)

In [0]:
reg.score(X_test, y_test)

0.6880526462127501

In [0]:
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR


optRF = BayesSearchCV(RandomForestRegressor(), {'n_estimators': [int(x) for x in np.linspace(start = 20, stop = 200, num = 10)], 'max_depth': [int(x) for x in np.linspace(10, 80, num = 8)]}, n_jobs=16)
optRF.fit(X_subset, y_subset)

/usr/local/lib/python3.6/dist-packages/skopt/learning/gaussian_process/gpr.py:294: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  X = check_array(X)
/usr/local/lib/python3.6/dist-packages/skopt/learning/gaussian_process/gpr.py:294: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  X = check_array(X)
/usr/local/lib/python3.6/dist-packages/skopt/learning/gaussian_process/gpr.py:294: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended th

BayesSearchCV(cv=None, error_score='raise',
              estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                              criterion='mse', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                 

In [0]:
optRF.best_params_

OrderedDict([('max_depth', 10), ('n_estimators', 180)])

In [0]:
rfStandard = RandomForestRegressor()
rfOpt = RandomForestRegressor(max_depth=10, n_estimators = 180)

In [0]:
rfStandard.fit(X_subset, y_subset)
rfOpt.fit(X_subset, y_subset)

In [0]:
print("Standard Random Forest Score: ", rfStandard.score(X_test, y_test), "  Random Forest with optimal parameters Score: ", rfOpt.score(X_test, y_test)

NotFittedError: ignored

In [0]:
optSVR = BayesSearchCV(SVR(kernel='rbf'), {'C': [0.1, 1, 10, 100], 'gamma': [0.1,0.01,0.001]}, n_jobs=16, n_iter=32, random_state=0)
optSVR.fit(X_subset, y_subset)

/usr/local/lib/python3.6/dist-packages/skopt/learning/gaussian_process/gpr.py:294: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  X = check_array(X)
/usr/local/lib/python3.6/dist-packages/skopt/learning/gaussian_process/gpr.py:294: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  X = check_array(X)
/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.6/dist-pac

BayesSearchCV(cv=None, error_score='raise',
              estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                            epsilon=0.1, gamma='scale', kernel='rbf',
                            max_iter=-1, shrinking=True, tol=0.001,
                            verbose=False),
              fit_params=None, iid=True, n_iter=32, n_jobs=16, n_points=1,
              optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=0,
              refit=True, return_train_score=False, scoring=None,
              search_spaces={'C': [0.1, 1, 10, 100],
                             'gamma': [0.1, 0.01, 0.001]},
              verbose=0)

In [0]:
optSVR.best_params_

OrderedDict([('C', 100), ('gamma', 0.001)])

In [0]:
svr = SVR(kernel='rbf', C=100, gamma=0.001)

NameError: ignored

In [0]:
svr.score(X_test, y_test)